In [7]:
import psycopg2
import psycopg2.extras

```sql
SELECT
  tablename.id,
  reverse(SPLIT_PART(SPLIT_PART(tablename.name, ',', numbers.n), ',', 1)) AS name
FROM
  (SELECT 1 n UNION ALL SELECT 2
   UNION ALL SELECT 3 UNION ALL SELECT 4) numbers INNER JOIN tablename
  ON CHARACTER_LENGTH(tablename.name)
     -CHARACTER_LENGTH(REPLACE(tablename.name, ',', ''))>=numbers.n-1
ORDER BY
  id, n
```

# <span style="color:Yellow">HIGH RISK SECRET</span>

In [23]:
with psycopg2.connect(
    database = "mercury",
    user = "human",
    host = '11.0.0.1',
    password = '@LearnBeyondTheBook!'
) as connection:
    
#     sysid_columns = ['SysId','Name','LastSeen','ClientAddr','UserDefinedClientName']
#     sql = f'''SELECT "{'","'.join(sysid_columns)}" FROM public."Sm2SysIdEntry" '''

#     crs = connection.cursor(cursor_factory = psycopg2.extras.NamedTupleCursor)
#     crs.execute(sql)
#     sysids = crs.fetchall()

#     project_columns = ['ProjectName', 'IsLiveCollaborationEnabled', 'SysIds', 'SM_Project_id']
#     sql = f'''SELECT "{'","'.join(project_columns)}" FROM public."SM_Project" '''

#     crs = connection.cursor(cursor_factory = psycopg2.extras.NamedTupleCursor)
#     crs.execute(sql)
#     projects = crs.fetchall()

    # Returns a flattened representation of SM_Project, where an entry like:
    # | project  |   sysids       |
    # | project1 | sysid1, sysid2 |
    # BECOMES:
    # | project  |   sysids       |
    # | project1 | sysid1         |
    # | project1 | sysid2         |
    
    crs = connection.cursor(cursor_factory = psycopg2.extras.NamedTupleCursor)
    
    sql = '''SELECT "SysIds" FROM public."SM_Project"'''
    crs.execute(sql)
    ret = crs.fetchall()
    
    max_users_in_project = 0
    
    for row in ret:
        users_in_project = len(row.SysIds.split(','))
        max_users_in_project = max(users_in_project, max_users_in_project)
        
    numbers = f"""(SELECT 1 n """
    
    for n in range(1,max_users_in_project):
        numbers += f'UNION ALL SELECT {n+1} '
        
    numbers += ")"

    sql = f"""SELECT
      public."SM_Project"."ProjectName",
      reverse(SPLIT_PART(SPLIT_PART(public."SM_Project"."SysIds", ',', numbers.n), ',', 1)) AS SysId
    FROM
      {numbers} numbers INNER JOIN public."SM_Project"
      ON CHARACTER_LENGTH(public."SM_Project"."SysIds")
         -CHARACTER_LENGTH(REPLACE(public."SM_Project"."SysIds", ',', ''))>=numbers.n-1
         
    ORDER BY
      "ProjectName", n
    """
    print(sql)
    
    crs.execute(sql)
    ret = crs.fetchall()    

SELECT
      public."SM_Project"."ProjectName",
      reverse(SPLIT_PART(SPLIT_PART(public."SM_Project"."SysIds", ',', numbers.n), ',', 1)) AS SysId
    FROM
      (SELECT 1 n ) numbers INNER JOIN public."SM_Project"
      ON CHARACTER_LENGTH(public."SM_Project"."SysIds")
         -CHARACTER_LENGTH(REPLACE(public."SM_Project"."SysIds", ',', ''))>=numbers.n-1
         
    ORDER BY
      "ProjectName", n
    


In [24]:
ret

[Record(ProjectName='Gödel', sysid='495BC9BF22C3')]